In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

ROOT_DIR = os.path.abspath('..')
sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from neuron.neurons import User, Neuron, ConversationalOrchestrator, ConversationalOrchestratorManager, UserTest
import os

c:\Users\FS-Ma\OneDrive\Documents\projects\neuron\.venv\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in CreateResult has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\FS-Ma\OneDrive\Documents\projects\neuron\.venv\Lib\site-packages\accelerate\utils\other.py:220: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.multiarray.
  np.core.multiarray._reconstruct,


In [3]:
llm_config = {
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "gemma2-9b-it",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

user = User(
    description="You are a human user interacting with conversational and recommendation agents."
)

conversational = Neuron(
    name="conversational",
    llm_config=llm_config,
    system_message="""
    "You are an expert conversational agent skilled at natural interactions. Whenever the user requests recommendations or suggestions, immediately involve the recommender agent.""",
    description="This agent acts as a social companion and facilitates interactions."
)

llm_config_2 = {
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "llama-3.3-70b-versatile",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

recommendation = Neuron(
    name="recommender",
    llm_config=llm_config_2,
    system_message="""
        You are an expert recommender agent. Whenever activated, you should clearly suggest the following items:
            1. Shoes
            2. T-shirts
            3. Jeans
Provide these suggestions directly when requested by the conversational agent or when the user explicitly asks for recommendations.""",
    description="This agent specializes in offering product recommendations."
)

chitchat = ConversationalOrchestrator(
    agents=[user, conversational, recommendation],
    messages=[],
    max_round=100,
    speaker_selection_method="random",
    send_introductions=True,
    allow_repeat_speaker = False,
)

chitchat_manager = ConversationalOrchestratorManager(chitchat=chitchat, llm_config=llm_config)

In [6]:
chat_result = user.initiate_chat(
    chitchat_manager, message="Oi tudo bem?", summary_method="reflection_with_llm"
)

message source='user' target='conversational_orchestrator' models_usage=None metadata={} content='Oi tudo bem?' type='TextMessage'
⟶ [user → conversational_orchestrator]:
Oi tudo bem?
message Response(chat_message=TextMessage(source='conversational_orchestrator', target=None, models_usage=RequestUsage(prompt_tokens=82, completion_tokens=14, total_tokens=96), metadata={}, content='\n\nPlease let us know how we can help you!\n', type='TextMessage'), inner_messages=[])
⟶ [conversational_orchestrator]:


Please let us know how we can help you!

